<a href="https://colab.research.google.com/github/prime29haruno/Kaggle_Titanic/blob/main/Titanic7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd, os

train = pd.read_csv("/kaggle/input/titanic/train.csv")
test  = pd.read_csv("/kaggle/input/titanic/test.csv")

# 欠損は「代入」で埋める（FutureWarning回避）
train["Age"] = train["Age"].fillna(train["Age"].median())
test["Age"]  = test["Age"].fillna(train["Age"].median())

# 性別を数値化（applyではなく map でOK）
test["Sex"] = test["Sex"].map({"male": 0, "female": 1})
test["Embarked"] = test["Embarked"].map({"S": 1, "Q": 2, "C": 3})

test["FamilySize"] = test["SibSp"] + test["Parch"] + 1

def rule_based_prediction(row):
    age = row["Age"]; sex = row["Sex"];
    pclass = row["Pclass"]; pid = row["PassengerId"];
    embarked = row["Embarked"]; familysize = row["FamilySize"];
    fare = row["Fare"]

    # 0=male, 1=female
    if pclass == 1:
        if sex == 1:
            if 0 <= age <= 12:
                return 0
            else:
                return 1
        else:
            if 0 <= age <= 12:
                return 1
            elif 19 <= age <= 35:
                if embarked == 3:
                    return 1
                else:
                    if 0 <= familysize <= 1:
                        if fare > 35.5:
                            return 0
                        else:
                            return 1
                    elif familysize >= 5:
                        return 0
                    else:
                        if fare > 61.769:
                            return 0
                        else:
                            return 1
            elif 36 <= age <= 50:
                if embarked == 3:
                    if 0 <= familysize <= 1:
                        return 0
                    else:
                        if 52.0 < fare <= 80.269:
                            return 1
                        else:
                            return 0
                elif embarked == 2:
                    return 0
                else:
                    if 0 <= familysize <= 1:
                        if fare > 27.428:
                            return 0
                        else:
                            return 1
                    else:
                        return 0

            elif 51 <= age:
                return 0
            else: #13-18歳
                if 109.396 >= fare:
                    return 0
                else:
                    return 1
    elif pclass == 2:
        if sex == 1:
            if 0 <= age <= 50:
                return 1
            else:
                    if 0 <= familysize <= 1:
                    return 0
                else:
                    return 1
        else:
            if 0 <= age <= 12:
                return 1
            else:
                return 0
    else:
        if sex == 1:
            if 36 <= age <= 50:
                return 0
            elif age >= 51:
                return 1
            elif 0 <= age <= 12:
                if 0 <= familysize <= 4:
                    return 1
                else:
                    return 0
            elif 13 <= age <= 18:
                if embarked == 2:
                        return 1
                elif embarked == 1:
                    if fare <= 7.75:
                        return 1
                    elif fare > 15.436:
                              return 1
                    else:
                        return 0
                else:
                    if 8.05 < fare <= 15.436:
                        return 0
                    else:
                        return 1
            else:
                if 5 <= familysize:
                    return 0
                else:
                    if embarked == 2:
                        return 1
                    elif embarked == 1:
                        if 8.69 < fare <= 14.454:
                            return 1
                        elif 6.749 < fare <= 7.746:
                            return 1
                        else:
                            return 0
                    else:
                        if 14.454 < fare <= 46.9:
                            return 0
                        else:
                            return 1
        else:
                  if 0 <= age <= 12:
                if pclass == 3:
                    if 0 <= familysize <= 1:
                        return 0
                    else:
                        return 1
                else:
                    return 0
            else:
                if pclass == 1:
                    if age >= 51:
                        return 0
                    else:
                        if 19 <= age <= 35:
                            if familysize >= 5:
                                return 0
                            else:
                                if embarked == 3:
                                    return 1
                                else:
                                    return 0
                        else:
                            if embarked == 3:
                                return 1
                            else:
                                return 0
                else:
                              return 0

test["Survived"] = test.apply(rule_based_prediction, axis=1)

# 提出ファイル作成（絶対に /kaggle/working に）
dst = "/kaggle/working/submission.csv"
test[["PassengerId","Survived"]].to_csv(dst, index=False)

print("作成OK?:", os.path.exists(dst), "→", dst)
print("working 内:", os.listdir("/kaggle/working"))
print(pd.read_csv(dst).head())